# Zadanie Domowe

2. Klasyfikacja grzybów (https://archive.ics.uci.edu/ml/datasets/Mushroom)
Przy użyciu danych mushrooms.csv zbuduj następujące modele: AdaBoost, XGBoost, LightGBM oraz CatBoost. Porównaj ich wyniki. Spróbuj ulepszyć wyniki każdego z wariantów poprzed dostrojenie parametrów - skorzystaj z dokumentacji wymienionych metod. 

In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [199]:
df = pd.read_csv("mushrooms.csv")

In [200]:
df.shape

(8124, 23)

In [201]:
df.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [203]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [204]:
df.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [205]:
features = df.drop("class", axis = 1)
features_dummies = pd.get_dummies(df.drop("class", axis = 1))
target = df["class"]

In [206]:
features_dummies.columns

Index(['cap-shape_b', 'cap-shape_c', 'cap-shape_f', 'cap-shape_k',
       'cap-shape_s', 'cap-shape_x', 'cap-surface_f', 'cap-surface_g',
       'cap-surface_s', 'cap-surface_y',
       ...
       'population_s', 'population_v', 'population_y', 'habitat_d',
       'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p', 'habitat_u',
       'habitat_w'],
      dtype='object', length=117)

In [207]:
target = target.replace(('e', 'p'), (1, 0))

In [208]:
target

0       0
1       1
2       1
3       0
4       1
       ..
8119    1
8120    1
8121    1
8122    0
8123    1
Name: class, Length: 8124, dtype: int64

In [209]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_dummies, target, test_size = 0.8, random_state = 1, stratify = target)

In [210]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [211]:
def evaluate(prediction, y_test): 
    result = classification_report(y_test, prediction, output_dict = True)
    f1 = result['1']['f1-score']
    accuracy = result['accuracy']
    performance_data= {'f1-score':round(f1, 4),
                      'accuracy':round(accuracy, 4)}
    return performance_data

## Logistic Regression

In [162]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 1)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_pr = evaluate(lr_pred, y_test)
lr_pr

{'f1-score': 0.9991, 'accuracy': 0.9991}

In [163]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

In [164]:
# l2 regularization + CV
from sklearn.model_selection import GridSearchCV
lr2 = LogisticRegression(random_state = 1)
lr2_cv = GridSearchCV(lr2, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}, cv = kf)
lr2_cv.fit(X_train, y_train)
print(lr2_cv.best_params_)
lr2_cv_pred = lr2_cv.predict(X_test)
lr2_cv_pr = evaluate(lr2_cv_pred, y_test)
lr2_cv_pr

{'C': 100}


{'f1-score': 1.0, 'accuracy': 1.0}

In [165]:
# l1 regularization + CV
lr3 = LogisticRegression(solver = 'liblinear', penalty = 'l1', random_state = 1)
lr3_cv = GridSearchCV(lr3, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}, cv = kf)
lr3_cv.fit(X_train, y_train)
print(lr3_cv.best_params_)
lr3_cv_pred = lr3_cv.predict(X_test)
lr3_cv_pr = evaluate(lr3_cv_pred, y_test)
lr3_cv_pr

{'C': 100}


{'f1-score': 1.0, 'accuracy': 1.0}

## Decision Tree

In [141]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state = 1)
dtree.fit(X_train, y_train)
dtree_pred = dtree.predict(X_test)
dtree_pr = evaluate(dtree_pred, y_test)
dtree_pr

{'f1-score': 0.9996, 'accuracy': 0.9995}

In [142]:
# Decision Tree + RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
param_rand = {'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10, 100, 10)}
dt_random = RandomizedSearchCV(estimator = DecisionTreeClassifier(random_state = 1), param_distributions = param_rand, n_iter = 100,
                               cv = kf, random_state = 1, n_jobs = -1)
dt_random.fit(X_train, y_train)
dt_random_pred = dt_random.predict(X_test)
dt_random_pr = evaluate(dt_random_pred, y_test)
dt_random_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [143]:
dt_cv.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [144]:
# Decision Tree + GridSearchCV
param_grid = {'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10, 100, 10)}
dt_cv = GridSearchCV(estimator = DecisionTreeClassifier(random_state=1), param_grid = param_grid, cv = kf, n_jobs = -1)
dt_cv.fit(X_train, y_train)
dt_cv_pred = dt_cv.predict(X_test)
dt_cv_pr = evaluate(dt_cv_pred, y_test)
dt_cv_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [145]:
dt_random.best_params_

{'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60}

In [146]:
# Decision Tree + BC
from sklearn.ensemble import BaggingClassifier
dtree_bc = BaggingClassifier(DecisionTreeClassifier(random_state = 1), random_state = 1)
dtree_bc.fit(X_train, y_train)
dtree_bc_pred = dtree_bc.predict(X_test)
dtree_bc_pr = evaluate(dtree_bc_pred, y_test)
dtree_bc_pr

{'f1-score': 0.9996, 'accuracy': 0.9995}

## Random Forest

In [147]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_pr = evaluate(rf_pred, y_test)
rf_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [148]:
# Random Forest + RandomizedSearchCV
param_rand = {'n_estimators': np.arange(100,800,100),
              'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1,2,4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10,100,10)}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state = 1), param_distributions = param_rand, n_iter = 100,
                               cv = kf, random_state = 1, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random_pred = rf_random.predict(X_test)
rf_random_pr = evaluate(rf_random_pred, y_test)
rf_random_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [149]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40}

In [150]:
# Random Forest + GridSearchCV - zajmuje duuużo czasu
# param_grid = {'n_estimators': np.arange(100,800,100),
#              'max_features' : ['auto', 'sqrt'],
#              'min_samples_leaf': [1,2,4],
#              'min_samples_split': [2, 5, 10],
#              'max_depth' : np.arange(10,100,10)}
# rf_cv = GridSearchCV(estimator = RandomForestClassifier(random_state = 1), param_grid = param_grid,
#                               cv = kf, n_jobs = -1)
# rf_cv.fit(X_train, y_train)
# rf_cv_pred = rf_cv.predict(X_test)
# rf_cv_pr = evaluate(rf_cv_pred, y_test)
# rf_cv_pr

In [151]:
# rf_cv.best_params_

## SVM

In [152]:
from sklearn.svm import SVC
svc = SVC(random_state = 1)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_pr = evaluate(svc_pred, y_test)
svc_pr

{'f1-score': 0.9991, 'accuracy': 0.9991}

In [153]:
# 'rbf' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc2 = SVC(random_state = 1)
svc2_cv = GridSearchCV(svc2, param_grid)
svc2_cv.fit(X_train, y_train)
print(svc2_cv.best_params_)
svc2_cv_pred = svc2_cv.predict(X_test)
svc2_cv_pr = evaluate(svc2_cv_pred, y_test)
svc2_cv_pr

{'C': 1, 'gamma': 0.1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [154]:
# 'linear' + CV
svc3 = SVC(kernel = 'linear', random_state = 1)
svc3_cv = GridSearchCV(svc3, {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]})
svc3_cv.fit(X_train, y_train)
print(svc3_cv.best_params_)
svc3_cv_pred = svc3_cv.predict(X_test)
svc3_cv_pr = evaluate(svc3_cv_pred, y_test)
svc3_cv_pr

{'C': 1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [155]:
# 'poly' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc4 = SVC(kernel = 'poly', random_state = 1)
svc4_cv = GridSearchCV(svc4, param_grid)
svc4_cv.fit(X_train, y_train)
print(svc4_cv.best_params_)
svc4_cv_pred = svc4_cv.predict(X_test)
svc4_cv_pr = evaluate(svc4_cv_pred, y_test)
svc4_cv_pr

{'C': 0.001, 'gamma': 1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [156]:
# 'sigmoid' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc5 = SVC(kernel = 'sigmoid', random_state = 1)
svc5_cv = GridSearchCV(svc5, param_grid)
svc5_cv.fit(X_train, y_train)
print(svc5_cv.best_params_)
svc5_cv_pred = svc5_cv.predict(X_test)
svc5_cv_pr = evaluate(svc5_cv_pred, y_test)
svc5_cv_pr

{'C': 100, 'gamma': 0.01}


{'f1-score': 1.0, 'accuracy': 1.0}

In [157]:
# all
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel':['rbf','linear','poly','sigmoid']}
svc_all_params = SVC(random_state = 1)
svc_all_params_cv = GridSearchCV(svc_all_params, param_grid)
clf = svc_all_params_cv.fit(X_train, y_train)
print(svc_all_params_cv.best_params_)
svc_all_params_cv_pred = svc_all_params_cv.predict(X_test)
svc_all_params_cv_pr = evaluate(svc_all_params_cv_pred, y_test)
svc_all_params_cv_pr

{'C': 0.001, 'gamma': 1, 'kernel': 'poly'}


{'f1-score': 1.0, 'accuracy': 1.0}

## KNN

In [158]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_pr = evaluate(knn_pred, y_test)
knn_pr

{'f1-score': 0.9997, 'accuracy': 0.9997}

In [159]:
accuracies = {}
for neighbor in np.arange(1, 10):
    knn = KNeighborsClassifier(n_neighbors = neighbor)
    knn.fit(X_train, y_train)
    accuracies[neighbor] = knn.score(X_test, y_test)
accuracies

{1: 1.0,
 2: 1.0,
 3: 0.9996923076923077,
 4: 1.0,
 5: 0.9986153846153846,
 6: 0.9986153846153846,
 7: 0.9981538461538462,
 8: 0.9983076923076923,
 9: 0.9981538461538462}

In [160]:
# KNN + CV
knn2 = KNeighborsClassifier()
knn_cv = GridSearchCV(knn2, {'n_neighbors': np.arange(1,10)}, cv = kf)
knn_cv.fit(X_train, y_train)
print(knn_cv.best_params_)
knn_cv_pred = knn_cv.predict(X_test)
knn_cv_pr = evaluate(knn_cv_pred, y_test)
knn_cv_pr

{'n_neighbors': 2}


{'f1-score': 1.0, 'accuracy': 1.0}

In [167]:
prediction_data = {'Model':['Logistic Regression', 'Logistic Regression - l2 regularization + CV', 'Logistic Regression - l1 regularization + CV', 'Decision Tree', 'Decision Tree + RandomizedSearchCV', 'Decision Tree + GridSearchCV', 'Decision Tree + BC', 'Random Forest', 'Random Forest + RandomizedSearchCV', 'SVM rbf', 'SVM rbf + CV', 'SVM linear + CV', 'SVM poly + CV', 'SVM sigmoid + CV', 'SVM all params CV','KNN', 'KNN + CV'],
                   'F1-score':[lr_pr['f1-score'], lr2_cv_pr['f1-score'], lr3_cv_pr['f1-score'], dtree_pr['f1-score'], dt_random_pr['f1-score'], dt_cv_pr['f1-score'], dtree_bc_pr['f1-score'], rf_pr['f1-score'], rf_random_pr['f1-score'], svc_pr['f1-score'], svc2_cv_pr['f1-score'], svc3_cv_pr['f1-score'], svc4_cv_pr['f1-score'], svc5_cv_pr['f1-score'], svc_all_params_cv_pr['f1-score'], knn_pr['f1-score'], knn_cv_pr['f1-score']],
                   'Accuracy':[lr_pr['accuracy'], lr2_cv_pr['accuracy'], lr3_cv_pr['accuracy'], dtree_pr['accuracy'], dt_random_pr['accuracy'], dt_cv_pr['accuracy'], dtree_bc_pr['accuracy'], rf_pr['accuracy'], rf_random_pr['accuracy'], svc_pr['accuracy'], svc2_cv_pr['accuracy'], svc3_cv_pr['accuracy'], svc4_cv_pr['accuracy'], svc5_cv_pr['accuracy'], svc_all_params_cv_pr['accuracy'], knn_pr['accuracy'], knn_cv_pr['accuracy']]
                    }
 
prediction_table = pd.DataFrame(prediction_data)
prediction_table

,Model,F1-score,Accuracy
0,Logistic Regression,0.9991,0.9991
1,Logistic Regression - l2 regularization + CV,1.0000,1.0000
2,Logistic Regression - l1 regularization + CV,1.0000,1.0000
3,Decision Tree,0.9996,0.9995
4,Decision Tree + RandomizedSearchCV,1.0000,1.0000
5,Decision Tree + GridSearchCV,1.0000,1.0000
6,Decision Tree + BC,0.9996,0.9995
7,Random Forest,1.0000,1.0000
8,Random Forest + RandomizedSearchCV,1.0000,1.0000
9,SVM rbf,0.9991,0.9991


## AdaBoost

In [168]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state = 1)
ada.fit(X_train, y_train)
ada.score(X_test,y_test)

1.0

In [169]:
accuracies = {}
for n_estimators in np.arange(1, 10):
    ada = AdaBoostClassifier(random_state = 1, n_estimators = n_estimators)
    ada.fit(X_train, y_train)
    accuracies[n_estimators] = ada.score(X_test, y_test)
accuracies

{1: 0.8856923076923077,
 2: 0.8856923076923077,
 3: 0.939076923076923,
 4: 0.9578461538461538,
 5: 0.9670769230769231,
 6: 0.9515384615384616,
 7: 0.9781538461538462,
 8: 0.9823076923076923,
 9: 0.9949230769230769}

## Gradient Boosting

In [170]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate = 0.01 , random_state = 1)
gbc.fit(X_train, y_train)
gbc.score(X_test,y_test)

0.9969230769230769

In [180]:
accuracies = {}
for learning_rate in [0.01,0.1,0.3,0.5]:
    gbc = GradientBoostingClassifier(random_state = 1, learning_rate = learning_rate)
    gbc.fit(X_train, y_train)
    accuracies[learning_rate] = gbc.score(X_test, y_test)
accuracies

{0.01: 0.9969230769230769, 0.1: 0.9990769230769231, 0.3: 1.0, 0.5: 1.0}

## XGBoost

In [549]:
!pip install xgboost

In [181]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(random_state = 1,learning_rate = 0.01)
xgbc.fit(X_train, y_train)
xgbc.score(X_test, y_test)

0.9969230769230769

In [182]:
accuracies = {}
for learning_rate in [0.01,0.1,0.3,0.5]:
    xgbc = xgb.XGBClassifier(random_state = 1, learning_rate = learning_rate)
    xgbc.fit(X_train, y_train)
    accuracies[learning_rate] = xgbc.score(X_test, y_test)
accuracies

{0.01: 0.9969230769230769,
 0.1: 0.9990769230769231,
 0.3: 0.9990769230769231,
 0.5: 0.9990769230769231}

In [176]:
# xgboost cross-validation
dmatrix = xgb.DMatrix(data = features_dummies, label = target)
params = {"objective":"reg:logistic", "max_depth":3}
cv_results = xgb.cv(dtrain=dmatrix, params=params, 
                  nfold=3, num_boost_round=5, 
                  metrics="auc", as_pandas=True, seed=123)
print(cv_results)

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.986594       0.000814       0.986061      0.002084
1        0.997903       0.000227       0.997836      0.000314
2        0.998389       0.000528       0.998272      0.000173
3        0.998764       0.000233       0.998736      0.000433
4        0.998774       0.000225       0.998766      0.000452


## LightGBM

In [212]:
!pip install lightgbm


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [213]:
import lightgbm as lgb

In [214]:
train_data = lgb.Dataset(X_train, label = y_train)
#define parameters
params = {'learning_rate':0.001}
model = lgb.train(params, train_data, 100) 
y_pred = model.predict(X_test)
for i in range(0, 6500):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else: 
        y_pred[i] = 0
y_pred = pd.DataFrame(y_pred)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_pred, y_test)**0.5)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002388 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 180
[LightGBM] [Info] Number of data points in the train set: 1624, number of used features: 90
[LightGBM] [Info] Start training from score 0.517857
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [215]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

1.0

## CatBoost

In [38]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 6.9 MB/s eta 0:00:0000:0100:01
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 3.4 MB/s eta 0:00:0000:0100:01
  Using cached tenacity-8.1.0-py3-none-any.whl (23 kB)


In [216]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(X_train, y_train, cat_features=([ 0,  1, 2, 3, 4, 10]), eval_set=(X_test, y_test))
model.score(X_test, y_test)

/var/folders/gf/t0gbh7m514j9kjx2g299ch6h0000gn/T/ipykernel_89169/2126895021.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(df.dtypes != np.float)[0]


Learning rate set to 0.035721
0:	learn: 0.6111615	test: 0.6126876	best: 0.6126876 (0)	total: 22.1ms	remaining: 22s
1:	learn: 0.5356613	test: 0.5372831	best: 0.5372831 (1)	total: 27.1ms	remaining: 13.5s
2:	learn: 0.4801153	test: 0.4827737	best: 0.4827737 (2)	total: 31.7ms	remaining: 10.6s
3:	learn: 0.4294016	test: 0.4322028	best: 0.4322028 (3)	total: 35.4ms	remaining: 8.81s
4:	learn: 0.3695427	test: 0.3722865	best: 0.3722865 (4)	total: 42.1ms	remaining: 8.38s
5:	learn: 0.3262883	test: 0.3292413	best: 0.3292413 (5)	total: 46.8ms	remaining: 7.76s
6:	learn: 0.2869234	test: 0.2897918	best: 0.2897918 (6)	total: 53.7ms	remaining: 7.61s
7:	learn: 0.2652624	test: 0.2679422	best: 0.2679422 (7)	total: 55.6ms	remaining: 6.89s
8:	learn: 0.2354376	test: 0.2382833	best: 0.2382833 (8)	total: 60ms	remaining: 6.61s
9:	learn: 0.2127208	test: 0.2151849	best: 0.2151849 (9)	total: 62.2ms	remaining: 6.16s
10:	learn: 0.1909506	test: 0.1933360	best: 0.1933360 (10)	total: 65ms	remaining: 5.85s
11:	learn: 0.1668

107:	learn: 0.0081694	test: 0.0090851	best: 0.0090851 (107)	total: 334ms	remaining: 2.76s
108:	learn: 0.0080847	test: 0.0089943	best: 0.0089943 (108)	total: 337ms	remaining: 2.75s
109:	learn: 0.0080545	test: 0.0089582	best: 0.0089582 (109)	total: 341ms	remaining: 2.76s
110:	learn: 0.0080306	test: 0.0089306	best: 0.0089306 (110)	total: 343ms	remaining: 2.75s
111:	learn: 0.0079581	test: 0.0088498	best: 0.0088498 (111)	total: 347ms	remaining: 2.75s
112:	learn: 0.0079581	test: 0.0088498	best: 0.0088498 (112)	total: 349ms	remaining: 2.74s
113:	learn: 0.0078781	test: 0.0087771	best: 0.0087771 (113)	total: 351ms	remaining: 2.73s
114:	learn: 0.0076328	test: 0.0085132	best: 0.0085132 (114)	total: 354ms	remaining: 2.72s
115:	learn: 0.0075302	test: 0.0084065	best: 0.0084065 (115)	total: 356ms	remaining: 2.71s
116:	learn: 0.0073231	test: 0.0082188	best: 0.0082188 (116)	total: 359ms	remaining: 2.71s
117:	learn: 0.0071980	test: 0.0080868	best: 0.0080868 (117)	total: 362ms	remaining: 2.71s
118:	learn

228:	learn: 0.0028264	test: 0.0035808	best: 0.0035808 (228)	total: 700ms	remaining: 2.35s
229:	learn: 0.0028264	test: 0.0035808	best: 0.0035808 (229)	total: 702ms	remaining: 2.35s
230:	learn: 0.0028131	test: 0.0035669	best: 0.0035669 (230)	total: 705ms	remaining: 2.35s
231:	learn: 0.0028083	test: 0.0035626	best: 0.0035626 (231)	total: 708ms	remaining: 2.34s
232:	learn: 0.0028083	test: 0.0035626	best: 0.0035626 (232)	total: 710ms	remaining: 2.34s
233:	learn: 0.0028083	test: 0.0035626	best: 0.0035626 (233)	total: 712ms	remaining: 2.33s
234:	learn: 0.0028083	test: 0.0035626	best: 0.0035626 (233)	total: 713ms	remaining: 2.32s
235:	learn: 0.0028083	test: 0.0035626	best: 0.0035626 (235)	total: 715ms	remaining: 2.31s
236:	learn: 0.0027673	test: 0.0035246	best: 0.0035246 (236)	total: 721ms	remaining: 2.32s
237:	learn: 0.0027674	test: 0.0035246	best: 0.0035246 (237)	total: 724ms	remaining: 2.32s
238:	learn: 0.0027673	test: 0.0035246	best: 0.0035246 (238)	total: 725ms	remaining: 2.31s
239:	learn

350:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.33s	remaining: 2.47s
351:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.34s	remaining: 2.46s
352:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.34s	remaining: 2.46s
353:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.34s	remaining: 2.45s
354:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.34s	remaining: 2.44s
355:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.35s	remaining: 2.44s
356:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.35s	remaining: 2.44s
357:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.35s	remaining: 2.43s
358:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.36s	remaining: 2.42s
359:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.36s	remaining: 2.42s
360:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.36s	remaining: 2.41s
361:	learn

467:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.69s	remaining: 1.92s
468:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.69s	remaining: 1.92s
469:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.91s
470:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.91s
471:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.9s
472:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.89s
473:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.89s
474:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.88s
475:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.7s	remaining: 1.88s
476:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.71s	remaining: 1.87s
477:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 1.71s	remaining: 1.86s
478:	learn: 0.0026

594:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.02s	remaining: 1.38s
595:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.02s	remaining: 1.37s
596:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.03s	remaining: 1.37s
597:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.03s	remaining: 1.37s
598:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.04s	remaining: 1.36s
599:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.04s	remaining: 1.36s
600:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.04s	remaining: 1.35s
601:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.04s	remaining: 1.35s
602:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.05s	remaining: 1.35s
603:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.05s	remaining: 1.34s
604:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.05s	remaining: 1.34s
605:	learn

727:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.36s	remaining: 883ms
728:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.37s	remaining: 880ms
729:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.37s	remaining: 876ms
730:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.37s	remaining: 872ms
731:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.37s	remaining: 869ms
732:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.38s	remaining: 866ms
733:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.38s	remaining: 863ms
734:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.38s	remaining: 859ms
735:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.38s	remaining: 856ms
736:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.39s	remaining: 852ms
737:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.39s	remaining: 850ms
738:	learn

846:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.81s	remaining: 507ms
847:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.81s	remaining: 504ms
848:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.81s	remaining: 500ms
849:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.81s	remaining: 497ms
850:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.82s	remaining: 493ms
851:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.82s	remaining: 490ms
852:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.82s	remaining: 486ms
853:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.82s	remaining: 483ms
854:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.83s	remaining: 479ms
855:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.83s	remaining: 476ms
856:	learn: 0.0026533	test: 0.0034021	best: 0.0034021 (313)	total: 2.83s	remaining: 472ms
857:	learn

0.9995384615384615